# Religious counselor with Claude LLM



## Overview

This notebook demonstrates the usecase of a an Claude-v2 LLM powered religious counselor. Using AWS Bedrock APIs this note demonstrates 
1. How a Claude-v2 instances can be given a personality - a religious and spiritual guide in this case, and  
2. How can we use RAG (Retrieval Augmented Generation) to make LLM model more aware by learning from religious text books on the fly


## Use Cases Demonstrated

1. **Chatbot with persona** - We demonstrate a chatbot with a designated persona - spiritual and religious counselor. We will pre-populate that context before asking user for inputs
2. **Contextual-aware chatbot** - We make our chatbot even more knowledgeable by providing it embeddings from religious text. This leads to more well thought out guidance based on "learnings" from religious books that the user may choose. For this demo we use publically available and well cited PDFs of the Holy Bible and Bhagvad Gita as religious scriptures for embedding
 Source: http://triggs.djvu.org/djvu-editions.com/BIBLES/DRV/Download.pdf",
 Source: "https://www.gita-society.com/bhagavad-gita-in-english-source-file.pdf",



## Setup
### Setup Step 1: Setup Bedrock APIs

Setup environment. Primarily using Amazon Bedrock APIs in this context


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import os
import sys
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["AWS_PROFILE"] = "wailing_mckinsey"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
  Using profile: wailing_mckinsey
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


### Setup Step 2: Chatbot on Claude-v2 using conversation chain API

 Conversational memory allows to retain context between subsequent conversations



In [3]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 2000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)



### Setup Step 3: Chat UX Utility

The following utility class allows us to interact with Claude in a chat based UX

In [4]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , hope that conversation helped you in a meaningful way!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking through that...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

#### Test Chat based on on Claude's base understanding of the religious text "Bhagvad Gita":
1. What are moral takeaways from Bhagvad Gita? We do not see an informative response


In [5]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## Usecase: Religious and a spiritual guide with even more context
We make our religious counselor chatbot more knowledgeable by augmenting itself with religious text books downloaded. In this use case we will apply a pattern called RAG (Retrieval Augmented Generation) and create **Titan Embeddings Model** to create vectors for model to get additional context on the fly

In [7]:
from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### Retrieve sited religious books as an example

In order to be able to use embeddings for real-time context, we download cited religious text, split it up into documents and feed it into FAISS vector store to extract embeddings

In [8]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "http://triggs.djvu.org/djvu-editions.com/BIBLES/DRV/Download.pdf",
    "https://www.gita-society.com/bhagavad-gita-in-english-source-file.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [9]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [10]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 2403 documents loaded is 3964 characters.
After the split we have 11683 documents more than the original 2403.
Average length among 11683 documents (after split) is 870 characters.


#### Generate FAISS Vector Embeddings

In order to be able to use embeddings for real-time context, we download cited religious text, split it up into documents and feed it into FAISS vector store to extract embeddings

In [11]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings
    )

    print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

vectorstore_faiss_aws: number of elements in the index=11683::


### Use Claude v2 to generate throughtful questions that are context aware 
These questions are then fed into a second LLM model for conversational chat UX. The LLM is context aware on previous conversation

In [12]:
# turn verbose to true to see the full logs and documents
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"Human": "\n\nHuman: ", "Assistant": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 1000})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    max_tokens_limit=2000
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: As an assistant you will be acting as a religious counselor and a spiritual guide. Your goal is to life advice to users. Give detailed and meaningful reponses <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

## Final Demo: Religious Counselor and Guide ChatBot POC

Now that that the model is embedded with additional context from the religious text (something that the user can customize). It gives more detailed and responsive answers

In [1]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

NameError: name 'ChatUX' is not defined